In [4]:
import numpy as np
from numpy.linalg import eig

In [78]:
def get_mat(file):
    mat = []
    answ = []
    with open(file) as file_handler:
        for line in file_handler:
            row = list(map(int, line.split(',')[:-1]))
            mat.append(row)
    return np.array(mat)

def get_v(column, size, k):
    v = np.zeros(size)
    v[k] = column[k]+np.sign(column[k])*np.sqrt(np.sum(column[k:]*column[k:]))
    for i in range(k+1, size):
        v[i] = column[i]
    return v
        

def get_H(column, size, k):
    v = get_v(column,size,k)[:, np.newaxis]
    return np.eye(size) - 2*np.dot(v,v.T)/np.dot(v.T,v)
    

def get_A(H,A0):
    return np.dot(H,A0)

def get_QR(A):
    size = A[:,1].size
    Q = np.eye(size)
    for i in range(size-1):
        H = get_H(A[:,i],A[:,i].size,i)
        A = get_A(H,A)
        Q = np.dot(Q,H)
    return Q,A

def solve_equation(A, i):
    size = A[:,1].size
    if i+1 < size:
        a12 = A[i][i + 1]
        a21 = A[i + 1][i]
        a22 = A[i + 1][i + 1]
    else:
        a12 = 0
        a21 = 0
        a22 = 0
    a11 = A[i][i]
    a = 1
    b = -a11 - a22
    c = a11 * a22 - a12 * a21
    D = b*b-4*a*c
    if D < 0:
        l1 = (-b+np.sqrt(complex(D)))/2
        l2 = (-b-np.sqrt(complex(D)))/2
    else:
        l1 = (-b+np.sqrt(D))/2
        l2 = (-b-np.sqrt(D))/2
    return np.array([l1,l2])


def have_complex_lambda(A, eps, i):
        Q, R = get_QR(A)
        A1 = np.dot(R,Q)
        lambda1 = solve_equation(A, i)
        lambda2 = solve_equation(A1, i)
        if np.all(abs(lambda1 - lambda2) <= eps) and isinstance(lambda1[0], complex) and isinstance(lambda2[0], complex):
            return True
        else:
            return False

def get_eig(A,eps,i,k,Qi):
    while True:
        Q, R = get_QR(A)
        A = np.dot(R,Q)
        k+=1
        Qi.append(Q)
        if np.sqrt(np.sum(A[i + 1:, i]*A[i + 1:, i])) <= eps:
            return A[i,i], False, A, k
        elif np.sqrt(np.sum(A[i + 2:, i]*A[i + 2:, i])) <= eps and have_complex_lambda(A, eps, i):
            return solve_equation(A,i), True, A, k
        
def QR(A, eps):
    eigs = []
    i = 0
    k = 0
    c = 0
    size = A[:,1].size
    Qi = []
    Q = np.eye(size)
    while i < size:
        eig = get_eig(A,eps,i,k, Qi)
        k = eig[3]
        if eig[1]:
            eigs.append(eig[0])
            i+=2
        else:
            eigs.append(eig[0])
            i+=1
        A = eig[2]
    for Q_ in Qi:
        Q = np.dot(Q,Q_)
    return Q,eigs,k

A = get_mat("test2.txt") 
print("A:\n", A)
Q, eigs,k = QR(A,0.1)
print("Количество итераций:\n", k)
print("Первый собственный вектор:\n", Q[0,:])
print("Собственные значения:\n", eigs)

A:
 [[ 1  2  5]
 [-8  0 -6]
 [ 7 -9 -7]]
Количество итераций:
 19
Первый собственный вектор:
 [-0.37956053 -0.36912596 -0.84833945]
Собственные значения:
 [-11.909787271701685, 7.223770556763123, -1.2785266959322061]


In [75]:
np.linalg.qr(A)

(array([[-0.09365858,  0.35034395, -0.9319266 ],
        [ 0.74926865, -0.59158426, -0.29769877],
        [-0.65561007, -0.72614543, -0.2070948 ]]),
 array([[-10.67707825,   5.71317345,  -0.37463432],
        [  0.        ,   7.23599676,  10.3842433 ],
        [  0.        ,   0.        ,  -1.42377675]]))

ModuleNotFoundError: No module named 'matrix'